In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.cloud import bigquery
import os
import chart_studio
import chart_studio.plotly as py

# Tell Google API where the credentials are
credential_path = ''
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path
client = bigquery.Client()

# 1. What Kind of Crimes Should I Be Most Worried About?

In [9]:
# Get a count of offenses from 2015 to 2020 for Chicago and Hyde Park, and filter out domestic cases
sql_chicago = """select primary_type, count(primary_type) as count
                from `bigquery-public-data.chicago_crime.crime` 
                where date between '2015-01-01' and '2020-12-31' and domestic = False
                group by 1;"""

sql_hp = """select primary_type, count(primary_type) as count
                from `bigquery-public-data.chicago_crime.crime` 
                where date between '2015-01-01' and '2020-12-31' and community_area = 41 and domestic = False
                group by 1;"""

chicago = client.query(sql_chicago).to_dataframe()
hyde_park = client.query(sql_hp).to_dataframe()

# Group crimes that are are
chicago.loc[chicago['count'] < float(chicago.quantile(0.6)), 'primary_type'] = 'OTHER OFFENSE'
hyde_park.loc[hyde_park['count'] < float(hyde_park.quantile(0.6)), 'primary_type'] = 'OTHER OFFENSE'

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=chicago['primary_type'], values=chicago['count'], name=""), 1, 1)
fig.add_trace(go.Pie(labels=hyde_park['primary_type'], values=hyde_park['count'], name=""), 1, 2)

# Use `hole` to create a donut-like pie chart
#fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="A Look At Crimes in Chicago and Hyde Park From 2015 to 2020",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Chicago', x=0.16, y=1, font_size=20, showarrow=False),
                 dict(text='Hyde Park', x=0.85, y=1, font_size=20, showarrow=False)])
fig.write_html('figures/fig1.html')

# 2. What Types of Theft Are Are Involved?

In [8]:
# Get a count of offenses from 2015 to 2020 for Chicago and Hyde Park, and filter out domestic cases
sql_chicago = """select description, count(description) as count
                from `bigquery-public-data.chicago_crime.crime` 
                where date between '2015-01-01' and '2020-12-31' and domestic=False and primary_type='THEFT'
                group by 1;"""

sql_hp = """select description, count(description) as count
                from `bigquery-public-data.chicago_crime.crime` 
                where date between '2015-01-01' and '2020-12-31' and community_area=41 and domestic=False and primary_type='THEFT'
                group by 1;"""

# Pass query to Google cloud
chicago = client.query(sql_chicago).to_dataframe()
hyde_park = client.query(sql_hp).to_dataframe()

# Group crimes that are low in percentage for better chart resolution
chicago.loc[chicago['count'] < float(chicago.quantile(0.6)), 'description'] = 'OTHERS'
hyde_park.loc[hyde_park['count'] < float(hyde_park.quantile(0.6)), 'description'] = 'OTHERS'

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=chicago['description'], values=chicago['count'], name=""), 1, 1)
fig.add_trace(go.Pie(labels=hyde_park['description'], values=hyde_park['count'], name=""), 1, 2)

# Add some frills to make things pretty
fig.update_layout(
    title_text="A Look At Crimes in Chicago and Hyde Park From 2015 to 2020",
    annotations=[dict(text='Chicago', x=0.16, y=1, font_size=20, showarrow=False),
                 dict(text='Hyde Park', x=0.85, y=1, font_size=20, showarrow=False)])

fig.show()

# 3. How do the most 'popoular' crimes evolve over time?

In [10]:
# Get data from 2015, filter for only the most popular crimes, then group by month-year for chicago
sql_chicago = """select 
            format_date('%Y-%m', datetime(date, "US/Central")) as date, 
            count(case when primary_type = 'THEFT' and domestic = False then 1 end) as theft,
            count(case when primary_type = 'BATTERY' and domestic = False then 1 end) as battery,
            count(case when primary_type = 'CRIMINAL DAMAGE' and domestic = False then 1 end) as criminal_damange,
            count(case when primary_type = 'ASSAULT' and domestic = False then 1 end) as asasult,
            count(case when primary_type = 'DECEPTIVE PRACTICE' and domestic = False then 1 end) as deceptive
            from `bigquery-public-data.chicago_crime.crime` 
        where date between '2015-01-01' and '2020-12-31' group by 1;"""

# Since the returned results are first selected by UTC then converted to Central, 2014 make it on there.
chicago = client.query(sql_chicago).to_dataframe().sort_values('date', ignore_index=True)
chicago = chicago[chicago['date'] >= '2015-01']

# Get data from 2015, filter for only the most popular crimes, then group by month-year for hyde park
sql_hyde_park = """select 
            format_date('%Y-%m', datetime(date, "US/Central")) as date, 
            count(case when primary_type = 'THEFT' and domestic = False then 1 end) as theft,
            count(case when primary_type = 'BATTERY' and domestic = False then 1 end) as battery,
            count(case when primary_type = 'CRIMINAL DAMAGE' and domestic = False then 1 end) as criminal_damange,
            count(case when primary_type = 'ASSAULT' and domestic = False then 1 end) as asasult,
            count(case when primary_type = 'DECEPTIVE PRACTICE' and domestic = False then 1 end) as deceptive
            from `bigquery-public-data.chicago_crime.crime` 
        where date between '2015-01-01' and '2020-12-31' and community_area = 41 group by 1;"""

# Get data from 2015, filter for only the most popular crimes, then group by month-year for hyde park
hyde_park = client.query(sql_hyde_park).to_dataframe().sort_values('date', ignore_index=True)
hyde_park = hyde_park[hyde_park['date'] >= '2015-01']

# Visualize with Plotly
fig1 = px.line(chicago, x='date', y=chicago.columns, 
              title='A Look of Most Prevalent Crimes in Chicago as a Function of Time',
             labels={'value': 'monthly count'})
fig1.write_html('figures/fig3a.html')
fig1.show()

# Visualize with Plotly
fig2 = px.line(hyde_park, x='date', y=hyde_park.columns, 
              title='A Look of Most Prevalent Crimes in Hyde Park as a Function of Time', labels={'value': 'monthly count'})

fig2.show()

# 4. Are the Most Prevalent Crimes Decreasing?

In [11]:
# Query from Google's Bigquery Public Data 
sql_chicago = """select 
            format_date('%Y', datetime(date, "US/Central")) as date, 
            count(case when primary_type = 'THEFT' and domestic = False then 1 end) as theft,
            count(case when primary_type = 'BATTERY' and domestic = False then 1 end) as battery,
            count(case when primary_type = 'CRIMINAL DAMAGE' and domestic = False then 1 end) as criminal_damange,
            count(case when primary_type = 'ASSAULT' and domestic = False then 1 end) as asasult,
            count(case when primary_type = 'DECEPTIVE PRACTICE' and domestic = False then 1 end) as deceptive
            from `bigquery-public-data.chicago_crime.crime` 
    where date between '2015-01-01' and '2020-12-31' group by 1;"""

# Pass query to Google Cloud
chicago = client.query(sql_chicago).to_dataframe().sort_values('date', ignore_index=True)
chicago = chicago[chicago['date'] >= '2015-01']
fig1 = px.bar(chicago, x='date', y=chicago.columns, 
              title='A Look of Most Prevalent Crimes in Chicago as a Function of Time',
              labels={'value': 'year count'}, barmode='group')
fig1.write_html('figures/fig4a.html')
fig1.show()

# Query from Google's Bigquery Public Data 
sql_hyde_park = """select 
            format_date('%Y', datetime(date, "US/Central")) as date, 
            count(case when primary_type = 'THEFT' and domestic = False then 1 end) as theft,
            count(case when primary_type = 'BATTERY' and domestic = False then 1 end) as battery,
            count(case when primary_type = 'CRIMINAL DAMAGE' and domestic = False then 1 end) as criminal_damange,
            count(case when primary_type = 'ASSAULT' and domestic = False then 1 end) as asasult,
            count(case when primary_type = 'DECEPTIVE PRACTICE' and domestic = False then 1 end) as deceptive
            from `bigquery-public-data.chicago_crime.crime` 
    where date between '2015-01-01' and '2020-12-31' and community_area = 41 group by 1;"""

hyde_park = client.query(sql_hyde_park).to_dataframe().sort_values('date', ignore_index=True)
hyde_park = hyde_park[hyde_park['date'] >= '2015-01']
fig2 = px.bar(hyde_park, x='date', y=hyde_park.columns, 
              title='A Look of Most Prevalent Crimes in Hyde Park as a Function of Time',
              labels={'value': 'year count'}, barmode='group')

fig2.show()

In [12]:
# Get a count of offenses from 2015 to 2020 for Chicago and Hyde Park, and filter out domestic cases
sql_chicago = """select description, count(description) as count
                from `bigquery-public-data.chicago_crime.crime` 
                where date between '2020-01-01' and '2020-12-31' and domestic=False and primary_type='THEFT'
                group by 1;"""

sql_hp = """select description, count(description) as count
                from `bigquery-public-data.chicago_crime.crime` 
                where date between '2020-01-01' and '2020-12-31' and community_area=41 and domestic=False and primary_type='THEFT'
                group by 1;"""

# Pass query to Google Cloud
chicago = client.query(sql_chicago).to_dataframe()
hyde_park = client.query(sql_hp).to_dataframe()

# Group crimes that are are
chicago.loc[chicago['count'] < float(chicago.quantile(0.6)), 'description'] = 'OTHERS'
hyde_park.loc[hyde_park['count'] < float(hyde_park.quantile(0.6)), 'description'] = 'OTHERS'

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=chicago['description'], values=chicago['count'], name=""), 1, 1)
fig.add_trace(go.Pie(labels=hyde_park['description'], values=hyde_park['count'], name=""), 1, 2)

fig.update_layout(
    title_text="A Look At Crimes in Chicago and Hyde Park In 2020",
    annotations=[dict(text='Chicago', x=0.16, y=1, font_size=20, showarrow=False),
                 dict(text='Hyde Park', x=0.85, y=1, font_size=20, showarrow=False)])

fig.show()

# 5. What Days of The Week Are Most Dangerous? 

In [13]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Query from Google's Bigquery Public Data 
sql_chicago = """select
            format_date('%a', datetime(date, "US/Central")) as day_of_week,
            count(case when primary_type in ('BATTERY', 'ASSAULT') and domestic = False then 1 end) as count
            from `bigquery-public-data.chicago_crime.crime` 
            where date between '2015-01-01' and '2020-12-31' group by 1;"""

chicago = client.query(sql_chicago).to_dataframe().set_index('day_of_week').reindex(days)

# Query from Google's Bigquery Public Data 
sql_hp = """select
            format_date('%a', datetime(date, "US/Central")) as day_of_week,
            count(case when primary_type in ('BATTERY', 'ASSAULT') and domestic = False then 1 end) as count
            from `bigquery-public-data.chicago_crime.crime` 
            where date between '2015-01-01' and '2020-12-31' and community_area = 41 group by 1;"""

hyde_park = client.query(sql_hp).to_dataframe().set_index('day_of_week').reindex(days)

# Create subplots: use 'domain' type for Pie subplot
fig1 = px.bar(chicago, x=chicago.index, y=chicago.columns, title='Battery and Assault Count in Chicago', labels={'value': 'count'})
fig2 = px.bar(hyde_park, x=hyde_park.index, y=hyde_park.columns, title='Battery and Assault Count in Hyde Park', labels={'value': 'count'})

fig1.show()
fig2.show()

# 6. What Times of the Day Are Most Dangerous in Terms of Violent Crime?

In [14]:
# Query from Google's Bigquery Public Data 
sql_chicago = """select
            format_date('%H', datetime(date, "US/Central")) as hour,
            count(case when primary_type in ('BATTERY', 'ASSAULT') and domestic = False then 1 end) as count
            from `bigquery-public-data.chicago_crime.crime` 
            where date between '2015-01-01' and '2020-12-31' group by 1;"""

chicago = client.query(sql_chicago).to_dataframe().sort_values('hour', ignore_index=True)

# Query from Google's Bigquery Public Data 
sql_hp = """select
            format_date('%H', datetime(date, "US/Central")) as hour,
            count(case when primary_type in ('BATTERY', 'ASSAULT') and domestic = False then 1 end) as count
            from `bigquery-public-data.chicago_crime.crime` 
            where date between '2015-01-01' and '2020-12-31' and community_area = 41 group by 1;"""

hyde_park = client.query(sql_hp).to_dataframe().sort_values('hour', ignore_index=True)

fig1 = px.bar(chicago, x='hour', y='count', title='Battery and Assault Count in Chicago by Hour')
fig2 = px.bar(hyde_park, x='hour', y='count', title='Battery and Assault Count in Hyde Park by Hour')

fig1.show()
fig2.show()

# 7. Which Places are Most Susceptible to Violent Crimes?

In [15]:
# Query from Google's Bigquery Public Data 
sql_chicago = """select location_description, 
            count(location_description) as count
            from `bigquery-public-data.chicago_crime.crime` 
            where date between '2015-01-01' and '2020-12-31' and primary_type in ('BATTERY', 'ASSAULT') and domestic = False
            group by 1;"""

chicago = client.query(sql_chicago).to_dataframe()

# Query from Google's Bigquery Public Data 
sql_hp = """select location_description,
            count(location_description) as count
            from `bigquery-public-data.chicago_crime.crime` 
            where date between '2015-01-01' and '2020-12-31' and community_area = 41
            and primary_type in ('BATTERY', 'ASSAULT') and domestic = False
            group by 1;"""

hyde_park = client.query(sql_hp).to_dataframe()

# Group rare locations
chicago.loc[chicago['count'] < float(chicago.quantile(0.9)), 'location_description'] = 'OTHER'
hyde_park.loc[hyde_park['count'] < float(hyde_park.quantile(0.9)), 'location_description'] = 'OTHER'

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=chicago['location_description'], values=chicago['count'], name=""), 1, 1)
fig.add_trace(go.Pie(labels=hyde_park['location_description'], values=hyde_park['count'], name=""), 1, 2)

fig.update_layout(
    title_text="A Look At Crimes in Chicago and Hyde Park From 2015 to 2020",
    annotations=[dict(text='Chicago', x=0.16, y=1, font_size=20, showarrow=False),
                 dict(text='Hyde Park', x=0.85, y=1, font_size=20, showarrow=False)])

fig.show()